In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
gamedata_df = pd.read_csv('/home/welced12/git/football_analytics/data/espn_gamedata2009-2016.csv')

In [3]:
gamedata_df.head(5)

,gameId,result,season,week,home,away,winner,home_score,away_score,OT
0,290910023,"PIT 13, TEN 10 (OT)",2009,1,TEN,PIT,PIT,10,13,1
1,290913001,"ATL 19, MIA 7",2009,1,MIA,ATL,ATL,7,19,0
2,290913004,"DEN 12, CIN 7",2009,1,DEN,CIN,DEN,12,7,0
3,290913005,"MIN 34, CLE 20",2009,1,MIN,CLE,MIN,34,20,0
4,290913011,"IND 14, JAX 12",2009,1,JAX,IND,IND,12,14,0


In [4]:
teams = gamedata_df['home'].unique()
teams

array(['TEN', 'MIA', 'DEN', 'MIN', 'JAX', 'DET', 'DAL', 'PHI', 'KC',
       'NYJ', 'WSH', 'SF', 'STL', 'CHI', 'BUF', 'SD', 'CAR', 'CIN', 'HOU',
       'OAK', 'NE', 'NO', 'ARI', 'TB', 'SEA', 'PIT', 'CLE', 'BAL', 'NYG',
       'IND', 'GB', 'ATL', nan, 'LAR'], dtype=object)

In [5]:
# Define some functions that will be part of ELO ratings
def win_prob(home_elo,away_elo):
    exponent = (home_elo + 65 - away_elo)/400
    home_prob = 1 / ( 10**( -1 * exponent ) + 1 )
    away_prob = 1 / ( 10**(  1 * exponent ) + 1 )
    return (home_prob,away_prob)

In [6]:
win_prob(1629,1631)

(0.5896835031399501, 0.41031649686004995)

In [7]:
win_prob(1423,1413)

(0.6062878237854281, 0.3937121762145718)

In [8]:
def update_elo(home_elo,away_elo,home_score,away_score):
    # Make sure everything is floats
    if home_score == 'unknown':
        return (0,0)
    
    home_elo = float(home_elo)
    away_elo = float(away_elo)
    home_score = float(home_score)
    away_score = float(away_score)
    
    # Get probabilities of home/away wins
    p_hw, p_aw = win_prob(home_elo,away_elo)
    
    # Given actual score, figure out how to update ratings
    
    # Big K is a scaling factor for adjustments based on margin of victory
    big_k = 20 * math.log1p( math.fabs(home_score - away_score) ) \
            * 2.2 / (2.2 + 0.001*math.fabs( p_hw - p_aw ))
    
    home_win = 0
    away_win = 1
    if home_score > away_score:
        home_win = 1
        away_win = 0
    elif home_score == away_score:
        home_win = 0.5
        away_win = 0.5
    
    home_adjustment = big_k * (home_win - p_hw)
    away_adjustment = big_k * (away_win - p_aw)
    
    return (home_adjustment,away_adjustment)

In [9]:
update_elo(1629,1631,20,17)

(11.375461470854237, -11.375461470854239)

In [10]:
update_elo(1423,1413,7,33)

(-39.96065401087477, 39.96065401087478)

In [11]:
# Set up dataframe to keep track of ELO history for each team
teams = [x for x in gamedata_df['home'].unique() if str(x) != 'nan']
startval = 1500
start_dict = {(2009,1):{tm:startval for tm in teams}}
elo_hist = pd.DataFrame( start_dict )
elo_hist.head(5)

,2009
,1
ARI,1500
ATL,1500
BAL,1500
BUF,1500
CAR,1500


In [12]:
def get_elo(team,year,week):
    if isinstance(team, str):
        return elo_hist.loc[team,(year,week)]
    if math.isnan(team):
        return 0

# Try and update each team based on games in a week.
def update_for_week(year,week):
    relevant_games = gamedata_df.loc[ (gamedata_df['season'] == year) &
                                      (gamedata_df['week'] == week) ]
#    print(relevant_games)

    # make dict for weekly elo change
    elo_delta = {}
    
    for index in relevant_games.index.values:
        game = relevant_games.loc[index]
#        print(game)
        home = game['home']
        away = game['away']
        home_score = game['home_score']
        away_score = game['away_score']
        
        # pull elo ratings from elo history
        home_elo = get_elo(home,year,week)
        away_elo = get_elo(away,year,week)
        
        # Run function to calculate change in elo ratings after the week
        elo_change = update_elo(home_elo,
                                away_elo,
                                home_score,
                                away_score )
        
        if not home_elo == 0:
            elo_delta[home] = elo_change[0]
            elo_delta[away] = elo_change[1]
        
    return elo_delta

In [13]:
# Try running the function for a particular week
for wk in range(1,18):
    
    # Add column for this week's rating adjustment
    wkly_update = update_for_week(2009,wk)
    deltaname = str(wk)+'d'
    elo_hist[(2009,deltaname)] = pd.Series(wkly_update)
    # Make adjustment for teams that didn't play zero
    elo_hist.fillna(0, inplace=True)
    
    # Make new column for next week's rating
    elo_hist[(2009,wk+1)] = elo_hist[(2009,wk)] + elo_hist[(2009,deltaname)]
    # Drop column for rating adjustment
    elo_hist.drop( (2009,deltaname), axis=1, inplace=True )

elo_hist.head(5)

2009                                                                   \
       1            2            3            4            5            6    
ARI  1500  1486.883097  1508.961224  1485.071398  1485.071398  1511.015703   
ATL  1500  1530.390363  1552.815534  1514.120058  1514.120058  1548.157929   
BAL  1500  1532.085869  1545.754017  1576.807128  1550.953701  1538.770775   
BUF  1500  1491.787364  1522.074534  1500.482246  1454.544394  1441.335955   
CAR  1500  1472.556572  1450.131401  1422.853829  1422.853829  1441.436417   

                                                                      \
              7            8            9            10           11   
ARI  1537.210078  1555.419196  1524.871258  1548.220232  1572.778873   
ATL  1572.657843  1535.643867  1517.397883  1542.509004  1509.786971   
BAL  1527.424577  1527.424577  1572.663498  1543.351485  1553.835703   
BUF  1455.543885  1475.788861  1454.584257  1454.584257  1414.758541   
CAR  1455.442728  1435.197752  1465.745690  1451.385668  1484.107701   

                                                                      \
              12           13           14           15           16   
ARI  1580.644402  1560.921191  1598.546410  1557.908318  1563.394003   
ATL  1492.922243  1504.982829  1483.152018  1479.317422  1492.460136   
BAL  1547.319745  1564.460391  1532.792451  1555.558232  1583.370256   
BUF  1400.237789  1442.472147  1429.612659  1442.981592  1434.014287   
CAR  1467.626570  1438.411555  1462.186138  1441.963216  1491.632151   

                               
              17           18  
ARI  1578.177895  1554.522625  
ATL  1526.767944  1538.086920  
BAL  1564.140164  1571.404229  
BUF  1399.706479  1454.950739  
CAR  1522.821500  1564.555977

In [22]:
for yr in range(2010,2018):
    # Try running the function for a particular week
    for wk in range(1,18):
        
        # if week 1, pull rating from last week of previous season
        # and regress to mean by a third once everything else works
        if wk == 1:
            elo_hist[(yr,1)] = elo_hist[(yr-1,18)] \
                               - 0.3 * (elo_hist[(yr-1,18)] - 1400)
            
#        print(elo_hist.head(5))
    
        # Add column for this week's rating adjustment
        wkly_update = update_for_week(yr,wk)
        deltaname = str(wk)+'d'
        elo_hist[(yr,deltaname)] = pd.Series(wkly_update)
        # Make adjustment for teams that didn't play zero
        elo_hist.fillna(0, inplace=True)
    
        # Make new column for next week's rating
        elo_hist[(yr,wk+1)] = elo_hist[(yr,wk)] + elo_hist[(yr,deltaname)]
        # Drop column for rating adjustment
        elo_hist.drop( (yr,deltaname), axis=1, inplace=True )

elo_hist.head(5)

2009                                                                   \
       1            2            3            4            5            6    
ARI  1500  1486.883097  1508.961224  1485.071398  1485.071398  1511.015703   
ATL  1500  1530.390363  1552.815534  1514.120058  1514.120058  1548.157929   
BAL  1500  1532.085869  1545.754017  1576.807128  1550.953701  1538.770775   
BUF  1500  1491.787364  1522.074534  1500.482246  1454.544394  1441.335955   
CAR  1500  1472.556572  1450.131401  1422.853829  1422.853829  1441.436417   

                                                            ...       \
              7            8            9            10     ...        
ARI  1537.210078  1555.419196  1524.871258  1548.220232     ...        
ATL  1572.657843  1535.643867  1517.397883  1542.509004     ...        
BAL  1527.424577  1527.424577  1572.663498  1543.351485     ...        
BUF  1455.543885  1475.788861  1454.584257  1454.584257     ...        
CAR  1455.442728  1435.197752  1465.745690  1451.385668     ...        

            2017                                                      \
              9            10           11           12           13   
ARI  1440.410550  1440.410550  1440.410550  1440.410550  1440.410550   
ATL  1496.213933  1496.213933  1496.213933  1496.213933  1496.213933   
BAL  1389.242082  1389.242082  1389.242082  1389.242082  1389.242082   
BUF  1387.524719  1387.524719  1387.524719  1387.524719  1387.524719   
CAR  1436.494135  1436.494135  1436.494135  1436.494135  1436.494135   

                                                                      
              14           15           16           17           18  
ARI  1440.410550  1440.410550  1440.410550  1440.410550  1440.410550  
ATL  1496.213933  1496.213933  1496.213933  1496.213933  1496.213933  
BAL  1389.242082  1389.242082  1389.242082  1389.242082  1389.242082  
BUF  1387.524719  1387.524719  1387.524719  1387.524719  1387.524719  
CAR  1436.494135  1436.494135  1436.494135  1436.494135  1436.494135  

[5 rows x 162 columns]

In [23]:
elo_hist.loc[:,(2017)]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
ARI,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550,1440.410550
ATL,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933,1496.213933
BAL,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082,1389.242082
BUF,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719,1387.524719
CAR,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135,1436.494135
CHI,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577,1266.877577
CIN,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847,1438.466847
CLE,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002,1216.905002
DAL,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740,1494.113740
DEN,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784,1466.719784


In [18]:
import json

In [24]:
with open("elo_history.json","w") as f:
    json.dump( elo_hist.to_json(orient='index'), f )